# PyTorch 初心者ハンズオン

## PytorchによるNNの訓練

ここでは、MNISTと呼ばれる、よく知られる手書き文字データ・セットによって、NNの流れを体感していただきましょう。

## ページ内目次

<ul>
    <li>
        <a href="#PyTorchによるNNの訓練">PyTorchによるNNの訓練</a>
        <ul>
            <li><a href="#PyTorchのインポート、MNISTデータセットの読み込み">PyTorchのインポート、MNISTデータセットの読み込み</a></li>
            <li><a href="#NNの構造定義">NNの構造定義</a></li>
            <li><a href="#NN訓練前の準備">NN訓練前の準備</a></li>
            <li><a href="#NNの訓練">NNの訓練</a></li>
        </ul>
    <li>
        <a href="#生成モデルによる出力の理解">生成モデルによる出力の理解</a>
        <ul>
            <li><a href="#訓練したモデルを用いた予測">訓練したモデルを用いた予測</a></li>
            <li><a href="#出力値による割当クラスの決定">出力値による割当クラスの決定</a></li>
        </ul>
    <li>
        <a href="#softmax関数の利用">softmax関数の利用</a>
        <ul>
            <li><a href="#Softmax関数を試してみる">Softmax関数を試してみる</a></li>
            <li><a href="#Softmax関数を試してみる（注意点）">Softmax関数を試してみる（注意点）</a></li>
            <li><a href="#ユーティリティ関数の定義">ユーティリティ関数の定義</a></li>
        </ul>
    </li>
</ul>

<hr>

## PyTorchによるNNの訓練


下記構造のNNを、 MNIST データセットで訓練していきます。

<img src="_images/mlp_class.png">

入力値として手書き文字画像を与え、出力値として正しいクラスラベルを得ることが最終的な訓練の目的です。 <br>
たとえば、 9 の手書き文字を入力したら、 9 というクラスラベルが出力されるよう、ニューラルネットワークを訓練します。

<hr>

## PyTorchのインポート、MNISTデータセットの読み込み

最初に、今回利用する各種モジュールをインポートしておきます。
* NumPy （数値演算ライブラリ）
* matplotlib （グラフ描画ライブラリ）
* chainer

In [0]:
import numpy as np

import matplotlib.pyplot as plt

import torch as t
import torchvision as tv

MNISTデータセット（訓練用データ、および評価用データ）をロードする

In [0]:
preprocess = tv.transforms.Compose([
                                    tv.transforms.ToTensor(),
])

In [0]:
trainset = tv.datasets.MNIST('~/tmp/mnist', 
                               train=True,
                               download=True,
                               transform=preprocess)

In [0]:
testset = tv.datasets.MNIST('~/tmp/mnist',
                              train=False,
                              download=True,
                              transform=preprocess)

<hr>

## NNの構造定義

In [0]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
device

In [0]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

では、入力が784次元、出力が10次元のNNをクラス定義します。

PyTorchの世界では、NNを構築するクラスはtorch.nn.Moduleを継承している必要があります。

In [0]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        # the size of the inputs to each layer will be inferred
        self.l1 = nn.Linear(784, 1000)    # n_in -> n_units
        self.l2 = nn.Linear(1000, 1000)  # n_units -> n_units
        self.l3 = nn.Linear(1000, 10)     # n_units -> n_out

    def forward(self, x):
        h = x.view(-1, 28*28) # (N, 1, 28, 28) -> (N, 784)
        # １層目
        h = F.relu(self.l1(h))
        # ２層目
        h = F.relu(self.l2(h))
        # 出力層
        return self.l3(h)

MPLのNNを変数宣言しておく。

In [0]:
model = MLP()
model

これで定義したNNを用いた分類器の完成です。


<hr>

## NN訓練前の準備

オプティマイザ（誤差最小化アルゴリズム）を準備します。

> 説明は省略

In [0]:
from torch import optim

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

イテレータを準備します。

このイテレータによって、batchsize単位で訓練サンプルが取得できます。  
ランダムに取得することもできますが、再現性がなくなるので講義である今回はOFFにしておきましょう。

> Chainerで言えばiteratorを返すupdaterみたいなモノ

In [0]:
batchsize=100
train_loader = t.utils.data.DataLoader(trainset,
                         batch_size=batchsize,
                         shuffle=False)

最後にlossを計算する関数を定義しておきます。

In [0]:
criterion = nn.CrossEntropyLoss()

<hr>

## NNの訓練

ここまでで、NNを訓練する準備ができました。

ここでの訓練内容は、 trainset データセットにある入力値（28x28画像を784次元にしたもの）および目標値（0〜9のクラスラベル）を用いて、新たな入力値に対する予測値を求めることです。

訓練処理を、下記の通り実行します。

* 訓練前に、第1層の重み情報を画像として表示します。  
   > NNでは、訓練前の重み情報はランダムに設定されます。  
   > これは先入観を表していると捉えてもらえればいいです。
* 訓練を実行します。
* 訓練後、第1層の重み情報を画像として表示します。

In [0]:
param_dict_before = model.state_dict()

１層目の重みを保存しておきます。

In [0]:
l1w_before = param_dict_before["l1.weight"]

In [0]:
import time

次のセルは実行表示が、しばらくの間 ```In [*]```となっているはずです。  
訓練には時間がかかるため、完了するまでしばらく待ちましょう。

> - CPUの場合、約3sec/枚 
> - GPUの場合、約1sec/枚

In [0]:
def dev_env(tensor):
      return "cuda" if tensor.is_cuda else "cpu"

In [0]:
epochs=20
model.train() # トレーニングモードに遷移
criterion = criterion.to(device)
model = model.to(device) # GPUへ転送
for epoch in range(epochs):
    running_loss = 0.0
    start = time.time()
    for i, (inputs, labels) in enumerate(train_loader):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            end = time.time()
            print('[{:d}, {:5d}] loss: {:.3f} (elapsed: {:.1f} [s] by {})'
                    .format(epoch + 1, i + 1, running_loss / 100, end-start, dev_env(outputs)))
            running_loss = 0.0
            start = time.time()
print('Finished Training')

学習結果はGPU側のメモリに保存されているため、取り出す場合にはモデルデータをCPU側に転送する必要がある。  
ここでは、明示的に`cpu`メソッドをコールすることで解決します。

In [0]:
model = model.cpu()

In [0]:
param_dict_after = model.state_dict()
l1w_after = param_dict_after["l1.weight"]

訓練前と訓練後の重み画像を比較してみましょう。  
この画像から直接読み取れることは少ないですが、訓練を通じてモデル内のパラメータが変化したことが読み取れれば十分です。

In [0]:
fig, ax = plt.subplots(1,2, figsize=(10, 10))
# 訓練前のL1重みを画像として表示
ax[0].set_title("Layer1's weight [BEFORE]")
ax[0].imshow(l1w_before)
# 訓練前のL1重みを画像として表示
ax[1].set_title("Layer1's weight [AFTER]")
ax[1].imshow(l1w_after)

plt.show()

<hr>

# 生成モデルによる出力の理解

<div style="border: 1px solid; padding: 10px">
<p>さきほど、MNISTデータセットを用いて手書き画像を分類するためのNNを訓練しました。</p>
<p>続けて、訓練したモデルによる予測を試みます。</p>
<ul><li>MNISTデータセットのテスト用サンプルデータを入力し、出力を得る</li>
<li>出力から、どのクラスに予測されたか確認する</li>
<li>サンプルの画像、目標値、予測値の関係を確認する</li>
</ul>
</div>

<hr>

## 訓練したモデルを用いた予測

今から予測フェイズになります。  
モデルを訓練モードから評価モードにしましょう。

In [0]:
model = model.eval()

test_index 番目のテストデータの画像をNNに入力し、出力 y を得る

In [0]:
test_index=99
target = testset[99]

In [0]:
input_tensor = target[0]
input_tensor.is_cuda

In [0]:
input_tensor.unsqueeze_(0)
input_tensor.size()

In [0]:
p = model(Variable(input_tensor))

<hr>

## 出力値による割当クラスの決定

予測結果が持つdataメンバの形に注意。  
複数の入力を受けて複数の出力を出すために、二次元配列になっている。

In [0]:
p.data.shape

二次元配列ということを念頭に入れてdata[0]でアクセスすること。


In [0]:
d = p.data[0]
softmax_d = F.softmax(d, dim=0)
data_volume = len(d)

fig, ax = plt.subplots(1,2, figsize=(10, 5))
# 確率グラフを表示
ax[0].set_title("Prediction")
ax[0].set_xticks(np.arange(0, data_volume, 1))
ax[0].bar(x=range(data_volume), height=softmax_d)
# 画像データを表示
ax[1].set_title("Image")
ax[1].imshow(target[0].reshape(28,28))

最も出力値が大きな要素を予測ラベルとして採用する


In [0]:
predicted_label = np.argmax(softmax_d)
predicted_label

<hr>
【演習】<br>

* 自由に test データセットからサンプルを選び、予測結果を確認してください。
* この予測は 100% 正解にはなりません。色々なサンプルデータを試すと、予測結果が間違っているものも見つかるはずです。

In [0]:
# 演習




<hr>

【演習】

* <a href="#NN訓練前の準備">NN訓練前の準備</a>まで戻り、パラメータを変えて訓練、および、ここまでの確認項目をやり直してみてください。
   * epoch=1, batchsize=60000 で訓練し、予測結果がどうなるかを確認してください。 <br> この設定では訓練による重みの調整回数が減少するため、特に予測ミスが発生しやすくなります。
   * ブラウザの別タブなどで実行することにより2回の実行を横に並べて見比べるとよいでしょう。
   * （ランダムに学習されるため、毎回違う画像になりますが） epoch によって重みデータ画像の雰囲気が変化するはずです。
   * epoch=1とepoch=20でNNが出力する値にどのような違いがありますか。
   * epoch が大きいほど、NNの出力値の各要素は 0 もしくは 1 に近づいていきます。これは何故ですか。


訓練済みネットワークの出力値はスケールや範囲がまちまちです。

モデル自体は、 one-hot encoding によって、非正解が0, 正解が 1 になるように訓練しますが、 0 や 1 の値が返されるとはかぎりません（今回の問題は簡単なため、ちょっと長く訓練すれば 0 や 1 に近くなりますが、実際にはマイナス値をふくむ、もっと不安定な値が返されることのほうが多いです）。このため、分類問題においてNNの出力値そのままでは扱いづらい、といえます。

Softmax関数は、NNの出力ユニットの値をおよそ 0.00 ~ 1.00 の範囲に押さえ込み、確率として扱える値に変換します。

<hr>

## Softmax関数を試してみる

In [0]:
y = t.tensor([1.0,1.0,1.0])
F.softmax(y, dim=0)

In [0]:
y  = t.tensor([1.0, 2.0, 7.0])
F.softmax(y)

In [0]:
y = t.tensor([1.0, -5.0, 2.0])
F.softmax(y)

## Softmax関数を試してみる（注意点）

One-hot encoding の値がどのような確率に変換されるかを確認する

In [44]:
y = t.tensor([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
F.softmax(y)

NameError: ignored